In [1]:
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm
import numpy as np

import warnings
warnings.filterwarnings("ignore")

c:\Users\86177\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
ret = pd.read_feather('ret.txt').set_index('Trdwnt')['2004-01':'2023-52']
cap = pd.read_feather('cap.txt').set_index('Trdwnt')['2004-01':'2023-52']
ma = pd.read_feather('MA.txt')

In [7]:
mret = (ret.T*cap.T).sum() / cap.T.sum()
X_data = pd.concat([mret.shift(1),
           mret.shift(2),
           mret.shift(-1),
           mret.shift(-2),
           mret],axis=1)

In [19]:
# Average ret
final_data = []
for i in tqdm(range(20)):
    year = str(2004+i)
    ret_year = ret[year+'-00':year+'-99']
    X_year = X_data[year+'-00':year+'-99']
    for code in ret_year.columns:
        olsdata = pd.concat([ret_year[code],X_year],axis=1)
        olsdata.dropna(inplace=True)
        if len(olsdata) > 35:
            cap_ = cap[code][year+'-00':year+'-99'].mean()
            w_l_0 = sm.OLS(olsdata[code],sm.add_constant(olsdata[[0,1,2,3,4]])).fit().resid
            w_l = np.log(w_l_0+1)
            n = len(w_l)
            # NC-SKEW
            NCSKEW = (- (n*((n-1)**1.5)*(w_l*w_l*w_l).sum())) / ( (n-1) * (n-2) * (((w_l*w_l).sum())**1.5) )
            # DUVOL
            w_l_u = w_l[w_l>0]
            w_l_d = w_l[w_l<0]
            DUVOL = np.log(w_l_d.std() / w_l_u.std())
            final_data.append([code,year,np.log(cap_),NCSKEW,DUVOL])
pd.DataFrame(final_data,columns=['code','year','cap','NCSKEW','DUVOL']).to_feather('risk.txt')

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [03:02<00:00,  9.13s/it]


In [12]:
risk = pd.read_feather('risk.txt')
risk['time'] = (risk['year'] >= '2015').astype(int)
info = pd.read_feather('info.txt')
info['list'] = (info['Listdt'] > '2014-05-09').astype(int)
info['list']
info['ind'] = [i[0] for i in info['Nnindcd']]
controls = info[['Stkcd','Markettype','list','ind']]
controls.columns = ['code','type','list','ind']
data = pd.merge(risk,controls).dropna()
data = pd.merge(data,ma,how='left')
data.fillna(0,inplace=True)

In [13]:
type_d = pd.get_dummies(data['type'],drop_first=True)
ind_d = pd.get_dummies(data['ind'])
ind_d.drop('O',axis=1,inplace=True)
year_d = pd.get_dummies(data['year'],drop_first=True)

In [115]:
# 标准
data_1 = data#[data['list']==0]
data_1 = pd.concat([data[data['type']==1],data[data['type']==4]])

x = pd.concat([data_1[['list','cap']],ind_d],axis=1,join='inner')
y = data_1['NCSKEW']
with open('out.txt','w') as f:
    print(sm.OLS(y,sm.add_constant(x)).fit().summary(),file = f)
print(sm.OLS(y,sm.add_constant(x)).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                 NCSKEW   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.960
Date:                Sun, 21 Apr 2024   Prob (F-statistic):            0.00741
Time:                        16:20:34   Log-Likelihood:                -54029.
No. Observations:               41828   AIC:                         1.081e+05
Df Residuals:                   41808   BIC:                         1.083e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0782      0.050     -1.570      0.1

In [23]:
# 沪深A主板
data_2 = pd.concat([data[data['type']==1],data[data['type']==4]])

x = pd.concat([data_2[['time','list','MA_r']],ind_d],axis=1,join='inner')
y = data_2['NCSKEW']

sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 NCSKEW   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.757
Date:                Sun, 21 Apr 2024   Prob (F-statistic):           3.95e-05
Time:                        14:27:39   Log-Likelihood:                -54020.
No. Observations:               41828   AIC:                         1.081e+05
Df Residuals:                   41807   BIC:                         1.083e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
time           0.0342      0.009      3.669      0.000       0.016       0.053
list          -0.0448      0.013     -3.384      0.001      -0.071      -0.019
MA_r          -0.0107      0.005     -2.320      0.020      -0.020      -0.002
A             -0.1421      0.037     -3.853      0.000      -0.214      -0.070
B             -0.1739      0.027     -6.556      0.000      -0.226      -0.122
C             -0.1886      0.008    -23.670      0.000      -0.204      -0.173
D             -0.1675      0.021     -8.019      0.000      -0.208      -0.127
E             -0.2329      0.026     -8.839      0.000      -0.285      -0.181
F             -0.1653      0.018     -9.175      0.000      -0.201      -0.130
G             -0.2373      0.023    -10.368      0.000      -0.282      -0.192
H             -0.0625      0.075     -0.829      0.407      -0.210       0.085
I             -0.2007      0.020    -10.016      0.000      -0.240      -0.161
J             -0.1797      0.024     -7.628      0.000      -0.226      -0.134
K             -0.2025      0.020    -10.148      0.000      -0.242      -0.163
L             -0.1460      0.034     -4.238      0.000      -0.214      -0.078
M             -0.0644      0.052     -1.240      0.215      -0.166       0.037
N             -0.1096      0.036     -3.060      0.002      -0.180      -0.039
P             -0.0611      0.082     -0.749      0.454      -0.221       0.099
Q             -0.2255      0.072     -3.139      0.002      -0.366      -0.085
R             -0.1591      0.037     -4.348      0.000      -0.231      -0.087
S             -0.1948      0.053     -3.653      0.000      -0.299      -0.090
==============================================================================
Omnibus:                     1511.516   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4105.431
Skew:                          -0.131   Prob(JB):                         0.00
Kurtosis:                       4.512   Cond. No.                         21.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [89]:
# 分时段
data_3 = pd.concat([data[data['type']==1],data[data['type']==4]])
#data_3 = data
data_3 = data_3[data_3['time'] == 1]

x = pd.concat([data_3[['MA','cap']],ind_d],axis=1,join='inner')
#y = data_3['NCSKEW']
y = data_3['DUVOL']

with open('out.txt','w') as f:
    print(sm.OLS(y,sm.add_constant(x)).fit().summary(),file = f)
print(sm.OLS(y,sm.add_constant(x)).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                  DUVOL   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.669
Date:                Sun, 21 Apr 2024   Prob (F-statistic):           0.000104
Time:                        16:09:39   Log-Likelihood:                -10038.
No. Observations:               23972   AIC:                         2.012e+04
Df Residuals:                   23952   BIC:                         2.028e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0686      0.033     -2.079      0.0